In [9]:

class Node:
    def __init__(self, operator, attributes, inputs, outputs, scope):
        self.operator = operator
        self.attributes = attributes
        self.inputs = inputs
        self.outputs = outputs
        self.scope = scope

    @property
    def operator(self):
        return self._operator

    @operator.setter
    def operator(self, operator):
        self._operator = operator.lower()

    @property
    def attributes(self):
        return self._attributes

    @attributes.setter
    def attributes(self, attributes):
        self._attributes = attributes

    @property
    def inputs(self):
        return self._inputs

    @inputs.setter
    def inputs(self, inputs):
        self._inputs = inputs

    @property
    def outputs(self):
        return self._outputs

    @outputs.setter
    def outputs(self, outputs):
        self._outputs = outputs

    @property
    def scope(self):
        return self._scope

    @scope.setter
    def scope(self, scope):
        self._scope = scope

    def __repr__(self):
        text = ', '.join([str(v) for v in self.outputs])
        text += ' = ' + self.operator
        if self.attributes:
            text += '[' + ', '.join(
                [str(k) + ' = ' + str(v)
                 for k, v in self.attributes.items()]) + ']'
        text += '(' + ', '.join([str(v) for v in self.inputs]) + ')'
        return text
    


In [10]:

class Variable:
    def __init__(self, name, dtype, shape=None):
        self.name = name
        self.dtype = dtype
        self.shape = shape

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name):
        self._name = name

    @property
    def dtype(self):
        return self._dtype

    @dtype.setter
    def dtype(self, dtype):
        self._dtype = dtype.lower()

    @property
    def shape(self):
        return self._shape

    @shape.setter
    def shape(self, shape):
        self._shape = shape

    @property
    def ndim(self):
        return len(self.shape)

    def size(self):
        return self.shape

    def dim(self):
        return self.ndim

    def __repr__(self):
        text = '%' + self.name + ': ' + self.dtype
        if self.shape is not None:
            text += '[' + ', '.join([str(x) for x in self.shape]) + ']'
        return text

In [11]:

class Graph:
    def __init__(self, name, variables, inputs, outputs, nodes):
        self.name = name
        self.variables = variables
        self.inputs = inputs
        self.outputs = outputs
        self.nodes = nodes

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name):
        self._name = name

    @property
    def variables(self):
        return self._variables

    @variables.setter
    def variables(self, variables):
        self._variables = variables

    @property
    def inputs(self):
        return self._inputs

    @inputs.setter
    def inputs(self, inputs):
        self._inputs = inputs

    @property
    def outputs(self):
        return self._outputs

    @outputs.setter
    def outputs(self, outputs):
        self._outputs = outputs

    @property
    def nodes(self):
        return self._nodes

    @nodes.setter
    def nodes(self, nodes):
        self._nodes = nodes

    def __repr__(self):
        text = self.name
        text += ' (' + '\n'
        text += ',\n'.join(['\t' + str(v) for v in self.inputs]) + '\n'
        text += '):' + '\n'
        text += '\n'.join(['\t' + str(x) for x in self.nodes]) + '\n'
        text += '\t' + 'return ' + ', '.join([str(v) for v in self.outputs])
        return text


Flatten.py

In [5]:
from collections import deque

import torch
import torch.nn as nn

__all__ = ['flatten', 'Flatten']


def flatten(inputs):
    queue = deque([inputs])
    outputs = []
    while queue:
        x = queue.popleft()
        if isinstance(x, (list, tuple)):
            queue.extend(x)
        elif isinstance(x, dict):
            queue.extend(x.values())
        elif isinstance(x, torch.Tensor):
            outputs.append(x)
    return outputs


class Flatten(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *args, **kwargs):
        outputs = self.model(*args, **kwargs)
        return flatten(outputs)


Trace.py


In [2]:
import warnings

import torch
import torch.jit

from torchprofile.utils.ir import Variable, Node, Graph

__all__ = ['trace']


def trace(model, args=(), kwargs=None):
    assert kwargs is None, 'Keyword arguments are not supported for now. ' \
                           'Please use positional arguments instead!'

    with warnings.catch_warnings(record=True):
        graph, _ = torch.jit._get_trace_graph(Flatten(model), args, kwargs)

    variables = dict()
    for x in graph.nodes():
        for v in list(x.inputs()) + list(x.outputs()):
            if 'tensor' in v.type().kind().lower():
                variables[v] = Variable(
                    name=v.debugName(),
                    dtype=v.type().scalarType(),
                    shape=v.type().sizes(),
                )
            else:
                variables[v] = Variable(
                    name=v.debugName(),
                    dtype=str(v.type()),
                )

    nodes = []
    for x in graph.nodes():
        node = Node(
            operator=x.kind(),
            attributes={
                s: getattr(x, x.kindOf(s))(s)
                for s in x.attributeNames()
            },
            inputs=[variables[v] for v in x.inputs() if v in variables],
            outputs=[variables[v] for v in x.outputs() if v in variables],
            scope=x.scopeName() \
                .replace('Flatten/', '', 1) \
                .replace('Flatten', '', 1),
        )
        nodes.append(node)

    graph = Graph(
        name=model.__class__.__module__ + '.' + model.__class__.__name__,
        variables=[v for v in variables.values()],
        inputs=[variables[v] for v in graph.inputs() if v in variables],
        outputs=[variables[v] for v in graph.outputs() if v in variables],
        nodes=nodes,
    )
    return graph


handlers.py

In [3]:
import numpy as np

__all__ = ['handlers']


def addmm(node):
    # [n, p] = aten::addmm([n, p], [n, m], [m, p], *, *)
    n, m = node.inputs[1].shape
    m, p = node.inputs[2].shape
    return n * m * p


def addmv(node):
    # [n] = aten::addmv([n], [n, m], [m], *, *)
    n, m = node.inputs[1].shape
    return n * m


def bmm(node):
    # [b, n, p] = aten::bmm([b, n, m], [b, m, p])
    b, n, m = node.inputs[0].shape
    b, m, p = node.inputs[1].shape
    return b * n * m * p


def matmul(node):
    if node.inputs[0].ndim == 1 and node.inputs[1].ndim == 1:
        # [] = aten::matmul([n], [n])
        n = node.inputs[0].shape[0]
        return n
    elif node.inputs[0].ndim == 1 and node.inputs[1].ndim == 2:
        # [m] = aten::matmul([n], [n, m])
        n, m = node.inputs[1].shape
        return n * m
    elif node.inputs[0].ndim == 2 and node.inputs[1].ndim == 1:
        # [n] = aten::matmul([n, m], [m])
        n, m = node.inputs[0].shape
        return n * m
    elif node.inputs[0].ndim == 2 and node.inputs[1].ndim == 2:
        # [n, p] = aten::matmul([n, m], [m, p])
        n, m = node.inputs[0].shape
        m, p = node.inputs[1].shape
        return n * m * p
    elif node.inputs[0].ndim == 1:
        # [..., m] = aten::matmul([n], [..., n, m])
        *b, n, m = node.inputs[1].shape
        return np.prod(b) * n * m
    elif node.inputs[1].ndim == 1:
        # [..., n] = aten::matmul([..., n, m], [m])
        *b, n, m = node.inputs[0].shape
        return np.prod(b) * n * m
    else:
        # [..., n, p] = aten::matmul([..., n, m], [..., m, p])
        *b, n, p = node.outputs[0].shape
        *_, n, m = node.inputs[0].shape
        *_, m, p = node.inputs[1].shape
        return np.prod(b) * n * m * p


def mul(node):
    os = node.outputs[0].shape
    return np.prod(os)


def convolution(node):
    if node.outputs[0].shape[1] == node.inputs[1].shape[0]:
        oc, ic, *ks = node.inputs[1].shape
    else:
        ic, oc, *ks = node.inputs[1].shape
    os = node.outputs[0].shape
    return np.prod(os) * ic * np.prod(ks)


def batch_norm(node):
    # TODO: provide an option to not fuse `batch_norm` into `linear` or `conv`
    return 0


def instance_norm_or_layer_norm(node):
    os = node.outputs[0].shape
    return np.prod(os)


def avg_pool_or_mean(node):
    os = node.outputs[0].shape
    return np.prod(os)


handlers = (
    ('aten::addmm', addmm),
    ('aten::addmv', addmv),
    ('aten::bmm', bmm),
    ('aten::matmul', matmul),
    ((
        'aten::mul',
        'aten::mul_',
    ), mul),
    ('aten::_convolution', convolution),
    ('aten::batch_norm', batch_norm),
    ((
        'aten::instance_norm',
        'aten::layer_norm',
    ), instance_norm_or_layer_norm),
    ((
        'aten::adaptive_avg_pool1d',
        'aten::adaptive_avg_pool2d',
        'aten::adaptive_avg_pool3d',
        'aten::avg_pool1d',
        'aten::avg_pool2d',
        'aten::avg_pool3d',
        'aten::mean',
    ), avg_pool_or_mean),
    ((
        'aten::adaptive_max_pool1d',
        'aten::adaptive_max_pool2d',
        'aten::adaptive_max_pool3d',
        'aten::add',
        'aten::add_',
        'aten::alpha_dropout',
        'aten::cat',
        'aten::chunk',
        'aten::clone',
        'aten::constant_pad_nd',
        'aten::contiguous',
        'aten::div',
        'aten::div_',
        'aten::dropout',
        'aten::dropout_',
        'aten::embedding',
        'aten::eq',
        'aten::feature_dropout',
        'aten::flatten',
        'aten::gt',
        'aten::hardtanh_',
        'aten::int',
        'aten::lt',
        'aten::log_softmax',
        'aten::max_pool1d',
        'aten::max_pool1d_with_indices',
        'aten::max_pool2d',
        'aten::max_pool2d_with_indices',
        'aten::max_pool3d',
        'aten::max_pool3d_with_indices',
        'aten::max_unpool1d',
        'aten::max_unpool2d',
        'aten::max_unpool3d',
        'aten::ne',
        'aten::reflection_pad1d',
        'aten::reflection_pad2d',
        'aten::reflection_pad3d',
        'aten::relu',
        'aten::relu_',
        'aten::replication_pad1d',
        'aten::replication_pad2d',
        'aten::replication_pad3d',
        'aten::rsub',
        'aten::select',
        'aten::sigmoid',
        'aten::size',
        'aten::slice',
        'aten::softmax',
        'aten::softshrink',
        'aten::squeeze',
        'aten::sub',
        'aten::sum',
        'aten::t',
        'aten::tanh',
        'aten::threshold',
        'aten::transpose',
        'aten::view',
        'aten::zeros',
        'prim::constant',
        'prim::listconstruct',
        'prim::listunpack',
        'prim::numtotensor',
    ), None),
)


profile.py

In [23]:
import warnings

from torchprofile.utils.trace import trace

__all__ = ['profile_macs']


def profile_macs(model, args=(), kwargs=None, reduction=sum):
    results = dict()

    graph = trace(model, args, kwargs)
    for node in graph.nodes:
#         print(node)
        for operators, func in handlers:
            if isinstance(operators, str):
                operators = [operators]
            if node.operator in operators:
                if func is not None:
                    print(node.operator)
                    results[node] = func(node)
                break
        else:
            warnings.warn('No handlers found: "{}". Skipped.'.format(
                node.operator))

    if reduction is not None:
        return reduction(results.values())
    else:
        return results


In [24]:
import torch
from torchvision import models
import numpy as np
if __name__ == '__main__':
    for name, model in models.__dict__.items():
        if not name.islower() or name.startswith('__') or not callable(model):
            continue

        model = model().eval()
#       
        if 'resnet50' in name:
            if 'inception' not in name:
                inputs = torch.randn(1, 3, 224, 224)
            else:
                inputs = torch.randn(1, 3, 299, 299)
            macs = profile_macs(model, inputs)
#             graph = trace(model,inputs)
#             for node in graph.nodes:
#                 print(node)
#             print(graph)
            # with open('graph_resnet18.txt', 'w') as file:
            #     file.write(graph)
#         print('{}: {:.4g} G'.format(name, macs / 1e9))

aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::batch_norm
aten::_convolution
aten::bat

Yaml Loaders - utils.py


In [2]:
import glob
import os
import sys
from copy import deepcopy

import yaml
from yaml import dump

import yamlordereddictloader


class accelergy_loader(yaml.SafeLoader):
    """
    Accelergy yaml loader
    """

    def __init__(self, stream):

        self._root = os.path.split(stream.name)[0]
        super(accelergy_loader, self).__init__(stream)

def include_constructor(self, node):
    """
    constructor:
    parses the !include relative_file_path
    loads the file from relative_file_path and insert the values into the original file
    """
    filepath = self.construct_scalar(node)
    if filepath[-1] == ",":
        filepath = filepath[:-1]
    filename = os.path.join(self._root, filepath)
    with open(filename, "r") as f:
        return yaml.load(f, accelergy_loader)


def includedir_constructor(self, node):
    """
    constructor:
    parses the !includedir relative_file_path
    loads the file from relative_file_path and insert the values into the original file
    """
    filepath = self.construct_scalar(node)
    if filepath[-1] == ",":
        filepath = filepath[:-1]
    dirname = os.path.join(self._root, filepath)
    yamllist = []
    for filename in glob.glob(dirname + "/*.yaml"):
        with open(filename, "r") as f:
            yamllist.append(yaml.load(f, accelergy_loader))
    return yamllist
    
    
yaml.add_constructor("!include", include_constructor, accelergy_loader)
yaml.add_constructor("!includedir", includedir_constructor, accelergy_loader)

In [4]:
class accelergy_dumper(yamlordereddictloader.SafeDumper):
    """ Accelergy yaml dumper """

    def ignore_aliases(self, _data):
        return True


def create_folder(directory):
    """
    Checks the existence of a directory, if does not exist, create a new one
    :param directory: path to directory under concern
    :return: None
    """
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("ERROR: Creating directory. " + directory)
        sys.exit()


def merge_dicts(dict1, dict2):
    merge_dict = deepcopy(dict1)
    merge_dict.update(dict2)
    return merge_dict


def write_yaml_file(filepath, content):
    """
    if file exists at filepath, overwite the file, if not, create a new file
    :param filepath: string that specifies the destination file path
    :param content: yaml string that needs to be written to the destination file
    :return: None
    """
    if os.path.exists(filepath):
        os.remove(filepath)
    create_folder(os.path.dirname(filepath))
    out_file = open(filepath, "a")
    out_file.write(dump(content, default_flow_style=False, Dumper=accelergy_dumper))


def get_yaml_format(content):
    return dump(content, default_flow_style=False, Dumper=accelergy_dumper)


def write_file(filepath, content):
    if os.path.exists(filepath):
        os.remove(filepath)
    create_folder(os.path.dirname(filepath))
    out_file = open(filepath, "a")
    out_file.write(content)


def remove_quotes(filepath):
    """
    :param filepath: file that needs to processed
    :return: None
    removes the quotes inside yaml files
    """
    if os.path.exists(filepath):
        new_content = ""
        f = open(filepath, "r")

        for line in f:
            if "'" in line:
                line = line.replace("'", "")
                new_content += line
        f.close()
        os.remove(filepath)
        newf = open(filepath, "w")
        newf.write(new_content)
        newf.close()


def ERROR_CLEAN_EXIT(*argv):
    msg_str = "ERROR: "
    for arg in argv:
        if type(arg) is not str:
            print(msg_str)
            print(arg)
            msg_str = ""
        else:
            msg_str += arg + " "
    print(msg_str)
    sys.exit(1)


def WARN(*argv):
    msg_str = "Warn: "
    for arg in argv:
        if type(arg) is not str:
            print(msg_str)
            print(arg)
            msg_str = ""
        else:
            msg_str += arg + " "
    print(msg_str)


def INFO(*argv):
    msg_str = "Info: "
    for arg in argv:
        if type(arg) is not str:
            print(msg_str)
            print(arg)
            msg_str = ""
        else:
            msg_str += arg + " "
    print(msg_str)


def ASSERT_MSG(expression, msg):
    if not expression:
        ERROR_CLEAN_EXIT(msg)


def add_functions_as_methods(functions):
    def decorator(Class):
        for function in functions:
            setattr(Class, function.__name__, function)
        return Class

    return decorator


def register_function(sequence, function):
    sequence.append(function)
    return function


def remove_brackets(name):
    """Removes the brackets from a component name in a list"""
    if "[" not in name and "]" not in name:
        return name
    if "[" in name and "]" in name:
        start_idx = name.find("[")
        end_idx = name.find("]")
        name = name[:start_idx] + name[end_idx + 1 :]
        name = remove_brackets(name)
        return name

Systemstate.py

In [ ]:
from accelergy.utils import *
class SystemState():
    def __init__(self):
        self.cc_classes = {}
        self.pc_classes = {}
        self.arch_spec = None
        self.hier_arch_spec = None
        self.ccs = {}
        self.pcs = {}
        self.action_counts = None
        self.plug_ins = []
        self.ERT = None
        self.ART = None
        self.parser_version = None
        self.flags = {}
        self.energy_estimations = None

    def set_flag_s(self, flag_name_val_dict):
        self.flags.update(flag_name_val_dict)

    def set_accelergy_version(self, version):
        self.parser_version = version

    def set_hier_arch_spec(self, arch_dict):
        ASSERT_MSG(self.hier_arch_spec is None, 'interpreted input arch is set')
        self.hier_arch_spec = arch_dict

    def set_arch_spec(self, arch_spec):
        ASSERT_MSG(self.arch_spec is None, 'architecture spec is already set')
        self.arch_spec = arch_spec

    def add_cc_class(self, cc_class):
        cc_class_name = cc_class.get_name()
        ASSERT_MSG(cc_class_name not in self.cc_classes, '%s compound class is already added'%(cc_class_name))
        self.cc_classes[cc_class_name] = cc_class

    def add_pc_class(self, pc_class):
        pc_class_name = pc_class.get_name()
        ASSERT_MSG(pc_class_name not in self.pc_classes, '%s primitive class is already added'%(pc_class_name))
        self.pc_classes[pc_class_name] = pc_class

    def add_cc(self, cc):
        cc_name = cc.get_name()
        ASSERT_MSG(cc_name not in self.ccs, '%s compound component is already added'%(cc_name))
        self.ccs[cc_name] = cc

    def add_pc(self, pc):
        pc_name = pc.get_name()
        ASSERT_MSG(pc_name not in self.ccs, '%s compound component is already added'%(pc_name))
        self.pcs[pc_name] = pc

    def add_plug_ins(self, plug_ins):
        ASSERT_MSG(type(plug_ins) is list, 'plug in objects need to be passed in as a list')
        self.plug_ins = plug_ins

    def set_ERT(self, ERT):
        self.ERT = ERT

    def set_ART(self, ART):
        self.ART = ART

    def set_action_counts(self, action_counts):
        self.action_counts = action_counts

    def set_energy_estimations(self, energy_estimations):
        self.energy_estimations = energy_estimations

action.py

In [6]:
from copy import deepcopy


class Action(object):
    """Action class"""
    def __init__(self, action_def_dict):

        self.action_def_dict = action_def_dict
        self.name = action_def_dict['name']

        self._arguments = None
        self.set_arguments(action_def_dict)

        # compound action contains action definitions in terms of subcomponents
        self._subcomponents = None
        self.set_subcomponents(action_def_dict)

        # repeat has the same meaning as action share
        if 'repeat' in action_def_dict:
            self._action_share = action_def_dict['repeat']
        elif 'action_share' in action_def_dict:
            self._action_share = action_def_dict['action_share']
        else:
            self._action_share = None
        # only compound actions will later set this property
        self._primitive_list = None

    def set_arguments(self, action_def_dict):
        if 'arguments' in action_def_dict:
            self._arguments = {}
            for arg_name, arg_range in action_def_dict['arguments'].items():
                self._arguments[arg_name] = arg_range

    def set_subcomponents(self, action_def_dict):
        if 'subcomponents' in action_def_dict:
            self._subcomponents = {}
            for subcomp in action_def_dict['subcomponents']:
                subcompActions = []
                for subcompAction in subcomp['actions']:
                    subcompActions.append(Action(subcompAction))
                self._subcomponents[subcomp['name']] = subcompActions

    def set_primitive_list(self, primitive_list):
        self._primitive_list = primitive_list

    def set_action_share(self, new_action_share):
        """update the parsed repeat/action_share value"""
        self._action_share = new_action_share

    def set_argument(self, new_arg_dict):
        """ update one or more argument name-val pairs"""
        self._arguments.update(new_arg_dict)

    def set_subcomps(self, defined_subcomps):
        self._subcomponents = defined_subcomps

    def get_name(self):
        return self.name

    def get_action_share(self):
        return self._action_share

    def get_arguments(self):
        return self._arguments

    def get_argument(self, arg_name):
        return self._arguments[arg_name]

    def get_subcomps(self):
        ASSERT_MSG(self._subcomponents is not None, 'action does not have defined subcomponents')
        return self._subcomponents

    def get_primitive_list(self):
        return self._primitive_list

    def get_action_info_as_dict(self):
        action_dict = {'name': self.name}
        if self._subcomponents is not None: action_dict['subcomponents'] = self._subcomponents
        if self._arguments is not None: action_dict['arguments'] = self._arguments
        return action_dict

    def get_subactions(self, subcompName):
        ASSERT_MSG(self._subcomponents is not None and subcompName in self._subcomponents,
                   'cannot find subactions associated with %s for action %s'%(subcompName, self.name))
        return self._subcomponents[subcompName]

    def get_arg_val(self, argName):
        ASSERT_MSG(argName in self._arguments, 'argument name %s is not associated with action %s'%(argName, self.name))
        return self._arguments[argName]

    def set_arg(self, arg_dict):
        self._arguments.update(arg_dict)

    def flatten_action_args_into_list(self, mappingDict):
        """ flatten an action into a list representing all possible argument value combinations"""

        args = self.get_arguments()
        if args is None:
            return [self]  # no arguments, no need to flatten

        # an action needs to be flattened into a list of actions with the same action name but different arg vals
        total_entries = 1
        argument_range_record = {}
        for arg_name, arg_range in args.items():
            ASSERT_MSG(type(arg_range) is str, '%s: argument value for action %s is not string, cannot parse range'%(arg_name,self.name))
            ASSERT_MSG('..' in arg_range, '%s: argument value for action %s is not range, cannot parse range'%(arg_name,self.name))
            new_arg_range = Action.map_arg_range_bounds(arg_range, mappingDict)[0]
            startIdx, endIdx = Action.parse_arg_range(new_arg_range)
            total_entries *= (endIdx - startIdx + 1)
            argument_range_record[arg_name] = (startIdx, endIdx)

        action_list = []
        for entry_idx in range(total_entries):
            offset = 1
            arg_def = {}
            for arg_name, range_record in argument_range_record.items():
                arg_range = range_record[1] - range_record[0] + 1
                arg_def[arg_name] = (entry_idx // offset) % arg_range + range_record[0]
                offset *= arg_range
            subcomp_list = []
            new_action = deepcopy(self); new_action._arguments = arg_def
            action_list.append(new_action)
        return action_list

    @staticmethod
    def parse_arg_range(arg_range):
        """ Parse the start index and end index for an argument range"""
        if type(arg_range) is not str or '..' not in arg_range:
            ERROR_CLEAN_EXIT('cannot parse the argument range specification: ', arg_range)
        split_sub_string = arg_range.split('..')
        start_idx = int(split_sub_string[0])
        end_idx = int(split_sub_string[1])
        return start_idx, end_idx

    @staticmethod
    def expand_action_to_list_with_arg_values(action_info):
        """flatten actions with arguments into list
           1) input action is fully defined with numerical ranges
           2) output list contains a list of actions each with a possible set of argument values
        """

        action_name = action_info['name']
        total_entries = 1
        argument_range_record = {}
        for argument_name, argument_range in action_info['arguments'].items():
            start_idx, end_idx = Action.parse_arg_range(argument_range)
            total_entries *= (end_idx - start_idx + 1)
            argument_range_record[argument_name] = (start_idx, end_idx)
        expanded_list = [{'name': action_name, 'arguments':{}} for i in range(total_entries)]
        # construct list of dictionaries that contain all the possible combination of argument values
        for entry_idx in range(total_entries):
            offset = 1
            for argument_name, range_record in argument_range_record.items():
                arg_range = range_record[1] - range_record[0] + 1
                expanded_list[entry_idx]['arguments'][argument_name] = \
                    (entry_idx // offset) % arg_range + range_record[0]
                offset *= arg_range
        return expanded_list

    @staticmethod
    def map_arg_range_bounds(arg_range_str, attributes_dict):
        """
        arguments for actions might have ranges that are specified in terms of it attributes
        parses the argument ranges in the format int/str..int/str, where str can be arithmetic operation

        :param arg_range_str: string that decribes the range of a compound action
        :param attributes_dict: attribute name-value pairs of the compound component
        :return: parsed argument range, whether there was binding
        """
        split_sub_string = arg_range_str.split('..')
        detect_arg_range_binding = False
        # process the start index
        try:
            start_idx = int(split_sub_string[0])
        except ValueError:
            op_type, op1, op2 = parse_expression_for_arithmetic(split_sub_string[0], attributes_dict)
            if op_type is not None:
                start_idx = process_arithmetic(op1, op2, op_type)
            else:
                if split_sub_string[0] not in attributes_dict:
                    ERROR_CLEAN_EXIT('cannot find mapping from', arg_range_str, 'to', attributes_dict)
                start_idx = attributes_dict[split_sub_string[0]]
            detect_arg_range_binding = True

        # process the end index
        try:
            end_idx = int(split_sub_string[1])
        except ValueError:
            op_type, op1, op2 = parse_expression_for_arithmetic(split_sub_string[1], attributes_dict)
            if op_type is not None:
                end_idx = process_arithmetic(op1, op2, op_type)
            else:
                if split_sub_string[1] not in attributes_dict:
                    ERROR_CLEAN_EXIT('cannot find mapping from', arg_range_str, 'to', attributes_dict)
                end_idx = attributes_dict[split_sub_string[1]]
            detect_arg_range_binding = True

        new_arg_range_str = str(start_idx) + '..' + str(end_idx)

        return new_arg_range_str, detect_arg_range_binding

    @staticmethod
    def parse_arg_range(arg_range):
        if type(arg_range) is not str or '..' not in arg_range:
            ERROR_CLEAN_EXIT('cannot parse the argument range specification: ', arg_range)
        split_sub_string = arg_range.split('..')
        start_idx = int(split_sub_string[0])
        end_idx   = int(split_sub_string[1])
        return start_idx, end_idx

subcomponent.py

In [7]:
class Subcomponent:
    def __init__(self, comp_def):
        self.dict_reprsentation = comp_def
        if 'attributes' not in self.dict_reprsentation:
            self.dict_reprsentation['attributes'] = {}

    def set_name(self, name):
        self.dict_reprsentation['name'] = name

    def get_name(self):
        return self.dict_reprsentation['name']

    def get_class_name(self):
        return self.dict_reprsentation['class']

    def get_attributes(self):
        return self.dict_reprsentation['attributes']


    def get_area_share(self):
        return self.dict_reprsentation['area_share']

    def add_new_attr(self, attr_dict):
        self.dict_reprsentation['attributes'].update(attr_dict)

    def set_area_share(self, area_share):
        self.dict_reprsentation['area_share'] = area_share

componenclass.py

In [8]:
from copy import deepcopy
from collections import OrderedDict

class ComponentClass:
    def __init__(self, class_dict):
        self._name = class_dict['name']
        self._default_attributes = deepcopy(class_dict['attributes'])

        self._actions = {}
        self.set_actions(class_dict['actions'])

        self._subcomponents = None
        if 'subcomponents' in class_dict:
            self._subcomponents = {}
            self.type = 'compound'
            for scomp in class_dict['subcomponents']: self._subcomponents[scomp['name']] = Subcomponent(scomp)
        else:
            self.type = 'primitive'
        self._primitive_type = class_dict['primitive_type'] if 'primitive_type' in class_dict else None

    def set_actions(self, action_list):
        ASSERT_MSG(type(action_list) is list,
                   '%s class description must specify its actions in list format'%(self.get_name()))
        for action in action_list:
            ASSERT_MSG('name' in action, '%s class actions must contain "name" keys'%(self.get_name()))
            self._actions[action['name']] = Action(action)

    #-----------------------------------------------------
    # Getters
    #-----------------------------------------------------
    def get_name(self):
        return self._name

    def get_default_attr_to_apply(self, obj_attr_name_list):
        attr_to_be_applied = OrderedDict()
        for attr_name, attr_val in self._get_default_attrs().items():
            if attr_val == "must_specify":
                ASSERT_MSG(attr_name in obj_attr_name_list,
                           "attributes %s for compound class %s must be specified in architecture description"
                           %(attr_name, self.get_name()))
            if attr_name not in obj_attr_name_list:
                attr_to_be_applied[attr_name] = attr_val
        return attr_to_be_applied

    def _get_default_attrs(self):
        return self._default_attributes

    def _get_attr_name_list(self):
        return list(self._default_attributes.keys())

    def _get_attr_default_val(self, attrName):
        ASSERT_MSG(attrName in self._default_attributes, 'Attribute %s cannot be found in class %s'%(attrName, self.get_name()))
        return self._default_attributes[attrName]

    def get_action_name_list(self):
        return list(self._actions.keys())

    def get_action(self, actionName):
        ASSERT_MSG(actionName in self._actions, '%s does not exist in class %s'%(actionName, self.get_name()))
        return self._actions[actionName]

    def get_subcomponents_as_dict(self):
        ASSERT_MSG(self._subcomponents is not None, 'component class %s does not have subcomponents' % self.get_name())
        return self._subcomponents

    def get_primitive_type(self):
        return self._primitive_type


Integrations.py

action_counts:
  local:
  - action_counts:
    - counts: 31306
      name: idle
    - arguments:
        address_delta: 2
        data_delta: 1
      counts: 1152
      name: fill
    - arguments:
        address_delta: 0
        data_delta: 0
      counts: 1150
      name: read
    - arguments:
        address_delta: 2
        data_delta: 1
      counts: 2
      name: read
    name: eyeriss_like.weights_glb


  subtree:
    - name: eyeriss_like
      attributes:
        technology: 40nm
      local:
        - name: weights_glb
          class: smartbuffer_SRAM
          attributes:
            memory_width: 64
            memory_depth: 1024
            n_banks: 2
        - name: shared_glb
          class: smartbuffer_SRAM
          attributes:
            memory_width: 64
            n_banks: 25
            bank_depth: 512
            memory_depth: bank_depth * n_banks
            n_buffets: 2
            update_fifo_depth: 2
        - name: ifmap_NoC
          class: XY_NoC
          attributes:
            datawidth: 16
            col_id_width: 5
        - name: weights_NoC
          class: XY_NoC
          attributes:
            datawidth: 64
        - name: psum_write_NoC
          class: XY_NoC
          attributes:
            datawidth: 64
        - name: psum_read_NoC
          class: XY_NoC
          attributes:
            datawidth: 64
            Y_X_wire_avg_length: 4mm
      subtree:
      - name: PE[0..167]
        attributes:
          memory_width: 16
        local:
          - name: ifmap_spad
            class: smartbuffer_RF
            attributes:
              memory_depth: 12
              buffet_manager_depth: 0
          - name: weights_spad
            class: smartbuffer_SRAM
            attributes:
              memory_depth: 224
              buffet_manager_depth: 0
          - name: psum_spad
            class: smartbuffer_RF
            attributes:
              memory_depth: 24
              buffet_manager_depth: 24
              update_fifo_depth: 2
          - name: mac
            class: intmac
            attributes:
              datawidth: 16

In [4]:
class Scheduling():
    def __init__(self,opts=None,hwdesc=None,constrainsts=None):
        total_cycles = 0
          
    def run(self, graph)
            """
            """
        for nodes in graph.nodes():
         # Also check how to schedule next node 
        node_memory_statistics(node)
        if(memory_bandwidth<maxconstraints.bandwidth):
            step_cycles = get_number_cycles(node, True )
        logger.save(node,step_cycles, read_access, write_access, memory_bandwidth, ismemorybottleneck)
        total_cycles+=step_cycles
    logger.save(total_cycles)
    
    def node_memory_statistics(self):
            read_access = 0
            write_access = 0
            memory_bandwidth = 0
            
            switch(node.type):
                {
                    case "conv" : 
                    
                    
                    default "" : 
                    
                    
                }
                
        return read_access, write_access, memory_bandwidth


    def get_number_cycles(self, ismemorybottleneck, ):
          if(!ismemorybottleneck):
            return (node.computational_expense//hwdesc)
             else:
                return (total_memory_accesses//memory_bandwidth)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [5]:
def runner():
    """
    Runs the Input Graph
    """
    logger = Logger()
    maxconstraints = defconstraints("max.yaml")  
    minconstraints = defconstraints("min.yaml")  
    if (hwdesc!=None):
        print("Mapping the Model on the Given Hardware")
    else:
        print("Generating the Hardware Description and Logging Statistics")
        
    scheduler = Scheduling(opts, hwdesc, constraints)
    scheduler.run()
    plugins = ['accelergy_ART','accelergy_ERT','cacti_memory','orion_noc','aladdin_compute']
    instatiate_plugins()
    
    logger.save_statistics(area)
    logger.save_statistics(energy)
    
def instantiate_plugins():
    
    

    

SyntaxError: unexpected EOF while parsing (<ipython-input-5-ff39986a8cad>, line 25)

In [6]:
class Logger():
    """
    """

    def __init__(self,filename):
        logging.save()

    def __save__():
        




SyntaxError: unexpected EOF while parsing (<ipython-input-6-7dbab20dca1a>, line 11)

In [7]:
def generatehw():
    """
    Generate Hardware Description Yaml File 
    ### Iterate over Generated Hardware Description

    """


In [8]:
class defconstraints():
        def __init__(self, yamlfile):
            self.yamlfile = yamlfile      
            self.parse_and_set_props()
        def parse_and_set_props():
            
        
            

SyntaxError: unexpected EOF while parsing (<ipython-input-8-bba73270b77b>, line 8)

In [9]:
class plugins():
    def __init__(self):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-9-6c1848a3f1e1>, line 3)

visualizer.py

In [13]:
def roofline_model():

def generate_execution_movie():
    
def genvideologger(logger):
    # Generate a Time Lapse Video 
    
    

    

IndentationError: expected an indented block (<ipython-input-13-b5fa24ac47b5>, line 3)

In [11]:


import logging

logger = logging.getLogger()


def init_logger(log_file=None, log_file_level=logging.NOTSET):
    log_format = logging.Formatter("[%(asctime)s %(levelname)s] %(message)s")
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(log_format)
    logger.handlers = [console_handler]
    logging.setFormatter

    if log_file and log_file != '':
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(log_file_level)
        file_handler.setFormatter(log_format)
        logger.addHandler(file_handler)

    return logger


In [12]:
import logging 

logging.info("Message to log")